## 1. 사전 준비

### 0. 필요 라이브러리 호출

In [1]:
import datetime as dt, pandas as pd, numpy as np, datetime as dt
import pymysql, re, os
from pathlib import Path
from tqdm import tqdm
import collections
import gspread
import time
import copy

from haversine import haversine
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import MinMaxScaler
from scipy.stats import pearsonr


def live_db_conn():
    conn = pymysql.connect(host='host', user='user', password='password',autocommit=True,cursorclass=pymysql.cursors.DictCursor, db = "database")
    return conn


def dev_db_conn():
    conn = pymysql.connect(host='host', user='user', password='password',autocommit=True,cursorclass=pymysql.cursors.DictCursor)
    return conn


pd.set_option('display.max_columns', None)

import warnings

warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)
warnings.simplefilter(action='ignore', category=UserWarning)

### 1. 메뉴 사전

In [2]:
conn = pymysql.connect(host='host', user='user', password='password', cursorclass = pymysql.cursors.DictCursor)
curs = conn.cursor()

sql = """
        SELECT * FROM dictionary.dic_menu;
      """
curs.execute(sql)
dictionary_menu_df = pd.DataFrame(curs.fetchall())

curs.close()
conn.close()

official_menu_df = dictionary_menu_df[dictionary_menu_df['SRC']!='레드테이블']
official_menu_df.head()

,SRC,KO,EN,EN_IS_USE,CN,CN_IS_USE,JP,JP_IS_USE,TW,TW_IS_USE
0,한국관광공사,전복초,Braised Abalone,TRUE,炖鲍鱼(酱炖),TRUE,アワビの煮付け,TRUE,醬燒鮑魚,TRUE
1,한국관광공사,전복조림,Braised Abalone,TRUE,炖鲍鱼,TRUE,アワビの煮付け,TRUE,醬燒鮑魚,TRUE
2,한국관광공사,전복정식,Abalone Set Menu,TRUE,鲍鱼套餐,TRUE,アワビ定食,TRUE,鮑魚套餐,TRUE
3,한국관광공사,전복전골,Abalone Hot Pot,TRUE,鲍鱼火锅,TRUE,アワビの寄せ鍋,TRUE,鮑魚火鍋,TRUE
4,한국관광공사,전복장아찌,Pickled Abalone,TRUE,鲍鱼酱菜,TRUE,アワビの漬物,TRUE,鮑魚醬菜,TRUE


### 2. 메뉴 제외 및 추가 리스트

In [3]:
ban_list = pd.read_excel('../refer/final_ban_list.xlsx').dropna()

ban_list = ban_list['name'].tolist()

add_list = pd.read_excel('../refer/final_add_list.xlsx').dropna()

add_list = add_list['메뉴'].tolist()

### 3. 대한민국 행정구역 데이터

In [4]:
bjd = pd.read_csv('../refer/법정동 정보.csv',encoding='cp949')[['bjd_nm','sd_nm','sgg_nm','emd_nm']]
bjd = bjd[bjd['emd_nm'].notnull()]

sd_list = bjd['sd_nm'].unique().tolist()

KORLOC_dic = {}

# 지역별 시군구 딕셔너리 생성
for sd in sd_list:
    
    if sd == '서울특별시':
        
        sgg_list = bjd[bjd['sd_nm'] == sd]['sgg_nm'].unique().tolist()

        KORLOC_dic[sd] = sgg_list

    elif sd == '세종특별자치시':

        emd_list = bjd[bjd['sd_nm'] == sd]['emd_nm'].unique().tolist()
        KORLOC_dic[sd] = emd_list

    elif sd == '강원도':

        sgg_list = bjd[bjd['sd_nm'] == sd]['sgg_nm'].unique().tolist()
        KORLOC_dic['강원특별자치도'] = sgg_list

    else:

        sgg_list = bjd[bjd['sd_nm'] == sd]['sgg_nm'].unique().tolist()
        KORLOC_dic[sd] = sgg_list

KORLOC_dic = {k:list(set([i.split(' ')[0] for i in v])) for k,v in KORLOC_dic.items()}

## 2. 알고리즘

##### 1. TF-IDF 순으로 정렬
##### 2. 해당 지역에 해당 메뉴를 판매하는 매장의 개수를 계산
##### 3. 본 메뉴를 최종 머스트잇으로 선정하기 위해, 2번의 합(매장 개수)에 따라 최종적으로 해당 메뉴를 포함하는 매장이 몇개 이상이어야 하는지를 정함 (임의로 데이터 보고 선정, 호진 주관 개입)
- 평균적으로 100개 미만인 지역들은 최소 매장 수 5개 / 100개 이상 200개 미만 10개 /200개 이상 600개 미만 15개 /600개 이상 ~ 40개
##### 4. 선정된 개수를 통해 해당 개수보다 많은 경우의 메뉴들만 최종 머스트잇으로 선정 

In [40]:
seoul_list = ['종로구', '중구', '용산구', '성동구', '광진구', '동대문구', '중랑구', '성북구', '강북구', '도봉구', '노원구', '은평구', '서대문구', '마포구', '양천구', '강서구', '구로구', '금천구', '영등포구', '동작구', '관악구', '서초구', '강남구', '송파구', '강동구']

conn = live_db_conn()
curs = conn.cursor()

sql = """
        SELECT * From must_eat;
      """
curs.execute(sql)
must_eat = pd.DataFrame(curs.fetchall())

curs.close()
conn.close()

must_eat_name = must_eat['name'].tolist()
ko_list = official_menu_df['KO'].tolist()

total_list = ko_list + must_eat_name + add_list
total_set = set(total_list) # remove duplicates
total_list = sorted(list(total_set),reverse=True,key=len)
total_list = [x.replace(' ','') for x in total_list]

# 메뉴 별 메뉴 포함 매장 딕셔너리
menu_store = {}
no_data_area = []
idx = 0
no_data_food = []
for province, areas in tqdm((KORLOC_dic.items())):
    idx += 1
    print('====================================')
    print('<{}. {}>'.format(idx, province))
    print('Part 1. 지역 별 메뉴 카운트')
    total = pd.DataFrame()

    for area in areas:
        
        total_location = province + ' ' + area
        conn = live_db_conn()
        curs = conn.cursor()

        # is_main, normal 수정: status가 hold이지만 네이버 지도에서는 정상 운영인 경우가 존재, main 메뉴가 너무 적음
        # 서울의 경우는 is_main 메뉴만
        if province in seoul_list:

            sql = """
            SELECT s.id, s.name as store_name, s.address, p.id as product_id, p.name as product_name
            FROM product p
            inner join store s
            on p.store_id = s.id
            where s.address like '%{}%'
            and p.name != ''
            and p.name is not null
            and p.is_main = 1
            """.format(total_location)

        else:
            
            sql = """
            SELECT s.id, s.name as store_name, s.address, p.id as product_id, p.name as product_name
            FROM product p
            inner join store s
            on p.store_id = s.id
            where s.address like '%{}%'
            and p.name != ''
            and p.name is not null
            """.format(total_location)

        curs.execute(sql)

        label_df = pd.DataFrame(curs.fetchall())

        curs.close()
        conn.close()

        if len(label_df) == 0:
            no_data_area.append(province + ' ' + area)
            continue

        else:
        
            label_df['product_name'] = label_df['product_name'].apply(lambda x:x.replace(' ',''))
            
            # 메뉴별 카운트 딕셔너리
            menu_count_break = {}
    
            for menu in total_list:
                
                menu_count_break[menu] = 0
                
                menu_store[total_location + ' ' + menu] = set()

            # 메뉴별 이미 한 매장에서 등록이 되었다면 중복카운트되지 않도록 하기 위한 딕셔너리     
            menu_state = {key:False for key in menu_count_break.keys()}

            store_product = zip(label_df['id'].values, label_df['product_name'].values)
            store_product_list = [[a,b] for a,b in store_product]

            sorted_list = sorted(store_product_list, key=lambda x:len(x[1]), reverse=True)
            
            for vals in sorted_list:
                e_b = True
                store_id = vals[0]
                menu = vals[1]
                
                for key in menu_count_break.keys(): # 공식 메뉴
                    # 공식 메뉴 이름이 매장 메뉴에 있으면
                    if key in menu:
                        # 근데 이미 이미 그 매장에서 등록이 되었다면(True)
                        if menu_state[key]:
                            #통과
                            continue

                        else:
                            # 메뉴 카운트 +1 해주고
                            menu_count_break[key]+=1
                            # 해당 메뉴를 포함하는 매장 딕셔너리에 추가
                            menu_store[total_location + ' ' + key].add(store_id)
                            # 해당 매장에서 해당 메뉴는 Count됐으므로 통과
                            menu_state[key]=True
                            e_b=False
                            break

                if e_b:
                    no_data_food.append(menu)

                menu_state = {key:False for key in menu_count_break.keys()}


            
            menu_count_break = {k:v for k,v in menu_count_break.items() if k not in ban_list}
            sorted_menu_dictionary = sorted(menu_count_break.items(), key = lambda item: item[1], reverse=True)
            
            loc_count = pd.DataFrame(sorted_menu_dictionary, columns=['메뉴','TF-IDF'])
            loc_count = loc_count[loc_count['TF-IDF'] != 0]
            
            temp = loc_count.transpose()
            temp = temp.rename(columns=temp.iloc[0]).iloc[1:]
            temp.index = [area]

            total = pd.concat([total,temp])

    

    print('Part 2. 지역 메뉴 TF-IDF')
    tf = total.fillna(0)
    tf_sum = tf.astype(bool).sum(axis = 0)
    # 문서 개수
    tf_len = len(tf_sum)
    # Inverse Document Frequency
    idf = np.log((tf_len) / (tf_sum+1))
    tfidf = tf * idf                      
    tfidf = tfidf / np.linalg.norm(tfidf, axis = 1, keepdims = True)
    trsp = tfidf.transpose()
    trsp.columns = tfidf.index

    excel_df = pd.DataFrame(columns=['지역','메뉴','TF-IDF','메뉴 포함 매장','메뉴 포함 매장 수','해당 지역 메뉴 포함 매장 수(공통)'])

    print('Part 3. 지역 메뉴 TF-IDF 및 포함 매장 수')
    for area in trsp.columns:
        
        total_location = province + ' ' + area
        tmp_df = pd.DataFrame(trsp.sort_values(by=area, ascending=False).head(50)[area])
        tmp_df['지역'] = total_location
        tmp_df['메뉴'] = tmp_df.index
        
        tmp_df['메뉴 포함 매장'] = tmp_df['메뉴'].apply(lambda x:menu_store[total_location + ' ' + x])
        tmp_df['메뉴 포함 매장 수'] = tmp_df['메뉴 포함 매장'].apply(lambda x:len(x))
        temp_set = set()

        for store in tmp_df[tmp_df['지역']==total_location]['메뉴 포함 매장']:
            temp_set.update(store)
            
        tmp_df['해당 지역 메뉴 포함 매장 수(공통)'] = len(temp_set)
        tmp_df['TF-IDF'] = tmp_df[area]
        tmp_df = tmp_df[tmp_df['메뉴 포함 매장'] != set()]
        tmp_df = tmp_df[['지역','메뉴','TF-IDF','메뉴 포함 매장','메뉴 포함 매장 수','해당 지역 메뉴 포함 매장 수(공통)']]
        tmp_df.reset_index(inplace=True)
        tmp_df.drop(columns='index',inplace=True)
        tmp_df.sort_values(by='메뉴 포함 매장 수', ascending=False,inplace=True)
        excel_df = pd.concat([excel_df,tmp_df],axis=0)

    final_df = pd.DataFrame(columns=excel_df.columns)
    
    print('Part 4. 지역 필터 적용 후 최종 데이터프레임 생성')
    for area in trsp.columns:

        filter_dic = {}

        temp_area = province + ' ' + area

        temp_df = excel_df[excel_df['지역']==temp_area]

        loc_total = temp_df['해당 지역 메뉴 포함 매장 수(공통)'].unique().tolist()[0]

        if loc_total < 100:

            filter_dic[temp_area] = 5  

        elif loc_total >= 100 and loc_total < 200:

            filter_dic[temp_area] = 10 

        elif loc_total >= 200 and loc_total < 600:

            filter_dic[temp_area] = 15

        else:
            filter_dic[temp_area] = 40


        loc_avg = temp_df['메뉴 포함 매장 수'].mean()

        final_filter_count = min(filter_dic[temp_area],loc_avg)

        temp_final = temp_df[temp_df['메뉴 포함 매장 수']>=final_filter_count]
    
        final_df = pd.concat([final_df,temp_final], axis=0)

    final_df['단위'] = final_df['지역'].apply(lambda x:x.split(' ')[0])

    print('====================================')
    final_df.to_excel('../0824_output/{}.xlsx'.format(province),index=False)

  0%|          | 0/17 [00:00<?, ?it/s]

<1. 서울특별시>
Part 1. 지역 별 메뉴 카운트
Part 2. 지역 메뉴 TF-IDF
Part 3. 지역 메뉴 TF-IDF 및 포함 매장 수
Part 4. 지역 필터 적용 후 최종 데이터프레임 생성


  6%|▌         | 1/17 [40:51<10:53:39, 2451.25s/it]

<2. 부산광역시>
Part 1. 지역 별 메뉴 카운트
Part 2. 지역 메뉴 TF-IDF


 12%|█▏        | 2/17 [51:40<5:47:47, 1391.18s/it] 

...

<16. 경상남도>
Part 1. 지역 별 메뉴 카운트
Part 2. 지역 메뉴 TF-IDF


 94%|█████████▍| 16/17 [2:18:04<06:32, 392.39s/it]

Part 3. 지역 메뉴 TF-IDF 및 포함 매장 수
Part 4. 지역 필터 적용 후 최종 데이터프레임 생성
<17. 제주특별자치도>
Part 1. 지역 별 메뉴 카운트
Part 2. 지역 메뉴 TF-IDF


100%|██████████| 17/17 [2:20:09<00:00, 494.67s/it]

Part 3. 지역 메뉴 TF-IDF 및 포함 매장 수
Part 4. 지역 필터 적용 후 최종 데이터프레임 생성


## 3. 결과 정리

### 지역별 Must-Eat 결과 구글 스프레드시트 업로드 자동화

In [41]:
def export_dataframe_to_google_sheet(worksheet, dataframe):

    try:
        
        # 데이터프레임 삽입
        worksheet.update([dataframe.columns.values.tolist()] + dataframe.values.tolist())

        # 첫 행 Bold체
        worksheet.format('1',{'textFormat': {'bold': True}})

        # 모든 행 가운데 정렬
        worksheet.format('1:{}'.format(len(temp)+1), {"horizontalAlignment": "CENTER"})

        worksheet.format('1', 
                        {"backgroundColor": {
                                            "red": 0.7,
                                            "green": 0.8,
                                            "blue": 0.7}}
                                            )
        
    except Exception as e:
        
        print(e)

In [42]:
# 해당 경로에 json 파일 위치해야함.
json_file_path = "stately-rock-392501-a2389e485ec7.json"
gc = gspread.service_account(json_file_path)
spreadsheet_url = "https://docs.google.com/spreadsheets/d/1kYttX_qWAVxqpUbPeiXpfMs9RUmyHlJKM7Jla7pb5r0/edit#gid=0"
doc = gc.open_by_url(spreadsheet_url)

path_dir = 'C:/Users/Redtable/Desktop/hojin/Korea Travel Card/Must-Eat/0824_output/'
 
file_list = os.listdir(path_dir)

for file in file_list:

    temp = pd.read_excel(path_dir + '/' + file)

    worksheet = doc.add_worksheet(title='{}'.format(file.split('.')[0]), rows='{}'.format(len(temp)), cols='{}'.format(len(temp.columns)))

    export_dataframe_to_google_sheet(worksheet, temp)

    time.sleep(5)